In [2]:
import yaml
import json
from tqdm import tqdm
import os

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import h5py

def check_raw_data(subjets_data, jet_index=0):
    print(f"\n--- Checking Raw Data for Jet {jet_index} ---")
    print(f"Number of subjets: {subjets_data['subjet_pt'].shape[-1]}")
    print(f"Subjet features: {list(subjets_data.keys())}")
    print(f"Number of indices per subjet: {subjets_data['particle_indices'].shape[-1]}")
    print(f"Sample subjet feature values: ")
    for name in subjets_data.keys():
        print(name, subjets_data[name][jet_index])

def check_model_input(model_input, batch_index=0):
    print(f"\n--- Checking Model Input for Batch Item {batch_index} ---")
    print(f"Input shape: {model_input.shape}")
    if len(model_input.shape) == 3:
        batch_size, num_subjets, feature_dim = model_input.shape
        print(f"Batch size: {batch_size}")
        print(f"Number of subjets: {num_subjets}")
        print(f"Feature dimension: {feature_dim}")
        print("\nFirst few values of the first subjet:")
        print(model_input[0, 0, :10])
    else:
        print("Unexpected shape for model input")

def inspect_indices(subjets, num_samples=5):
    print("\n--- Inspecting Subjet Indices ---")
    for i in range(min(num_samples, len(subjets['subjet_eta']))):
        print(f"\nSubjet {i}:")
        print(f"  pT: {subjets['subjet_pt'][i]:.2f}")
        print(f"  eta: {subjets['subjet_eta'][i]:.2f}")
        print(f"  phi: {subjets['subjet_phi'][i]:.2f}")
        print(f"  num_ptcls: {subjets['subjet_num_ptcls'][i]}")
        print(f"  Indices: {subjets['particle_indices'][i, :10]}...")  # Print first 10 indices
        print(f"  Number of indices: {len(subjets['particle_indices'][i])}")

def normalize_features(particle_features, feature_names, config):
    print(f"Normalizing particle features with shape: {particle_features.shape}")
    
    normalized_features = particle_features.clone()
    
    for i, feature_name in enumerate(feature_names):
        method = config["INPUTS"]["SEQUENTIAL"]["Particles"].get(feature_name, "none")
        print(f"Normalizing particle feature '{feature_name}' with method '{method}'")
        
        if method == "normalize":
            mean = particle_features[:, i].mean()
            std = particle_features[:, i].std()
            std = std if std > 1e-8 else 1.0
            normalized_features[:, i] = (particle_features[:, i] - mean) / std
        elif method == "log_normalize":
            normalized_features[:, i] = torch.log1p(particle_features[:, i])
      
    
    print(f"Normalized particle features shape: {normalized_features.shape}")
    
    return normalized_features

class DimensionCheckLayer(torch.nn.Module):
    def __init__(self, name, expected_dims):
        super().__init__()
        self.name = name
        self.expected_dims = expected_dims

    def forward(self, x):
        if len(x.shape) != self.expected_dims:
            print(f"WARNING: {self.name} has {len(x.shape)} dimensions, expected {self.expected_dims}")
        return x

class JetDataset(Dataset):
    """JetDataset class for loading and processing jet data from HDF5 files.

    Args:
        Dataset (_type_): _description_
    """
    def __init__(self, file_path, num_jets=None, transform=None, config=None, debug=False):
        print(f"Initializing JetDataset with file: {file_path}")
        
        with h5py.File(file_path, 'r') as hdf:
            print("Loading features and subjets from HDF5 file")
            self.particles = {name: hdf['particles'][name][:] for name in hdf['particles']}
            self.subjets = {name: hdf['subjets'][name][:] for name in hdf['subjets']}
            self.labels = hdf['labels'][:]
        
        self.transform = transform
        self.config = config
        self.debug = debug
        if self.debug:
            print(f"Raw dataset size: {self.labels.shape[0]} jets")
            print("Particle features shapes")
            for name in self.particles.keys():
                print(f"shape of {name}: {self.particles[name].shape}")
            print("Subjet features shapes")
            for name in self.subjets.keys():
                print(f"shape of {name}: {self.subjets[name].shape}")
        
        self.filter_good_jets()
        
        if num_jets is not None:
            if self.debug:
                print(f"Reducing number of jets to: {num_jets}")
            for name in self.particles.keys():
                self.particles[name] = self.particles[name][:num_jets]
                if self.debug:
                    print(f"shape of {name}: {self.particles[name].shape}")
            for name in self.subjets.keys():
                self.subjets[name] = self.subjets[name][:num_jets]
                if self.debug:
                    print(f"shape of {name}: {self.subjets[name].shape}")
        self.labels = self.labels[:num_jets]
        print(f"Final dataset size: {self.labels.shape[0]} jets")

    def filter_good_jets(self):
        """
        Filters jets based on the number of real subjets and updates the dataset to only include 'good' jets.
        
        Modifies:
        - Updates self.particles, self.subjets, and self.labels to only include jets with at least 10 real subjets.
        
        Prints:
        - The shape of particles and subjets after filtering.
        - The total number of good jets remaining after filtering.
        """
        if self.debug:
            print("Filtering good jets...")
        good_jet_indices = []
        
        for jet_idx in range(self.labels.shape[0]):
            num_real_subjets = self.get_num_real_subjets(jet_idx)
            # print(f"Jet {jet_idx}: {num_real_subjets} real subjets")
            if num_real_subjets >= 10:
                good_jet_indices.append(jet_idx)
        
        for name in self.particles.keys():
            self.particles[name] = self.particles[name][good_jet_indices]
            if self.debug:
                print(f"shape of {name}: {self.particles[name].shape}")
        for name in self.subjets.keys():
            self.subjets[name] = self.subjets[name][good_jet_indices]
            if self.debug:
                print(f"shape of {name}: {self.subjets[name].shape}")
        self.labels = self.labels[good_jet_indices]
        print(f"Filtered to {self.labels.shape[0]} good jets")
    
    def get_num_real_subjets(self, jet_idx):
        """
        Returns the number of real subjets in a given jet.
        Parameters:
        - jet_idx: the index of the jet to check
        Returns:
        - int: The number of real subjets.
        Example:
        >>> self.subjets['subjet_num_ptcls'][jet_idx] = [15, 10, 10, 9, 8, 0, 0, 0, ..., 0]
        >>> get_num_real_subjets(jet_idx)
        5
        """
        return sum(1 for subjet_num_ptcls in self.subjets['subjet_num_ptcls'][jet_idx] if subjet_num_ptcls > 0)

    def __len__(self):
        """
        Returns the length of the jets in the dataset.
        Returns:
            int: Number of jets in the dataset.
        """

        return self.labels.shape[0]

    def __getitem__(self, idx):
        """
        Retrieves the particle features, subjets, and masks for a given index and processes them.
        Args:
            idx (int): The index of the item to fetch.
        Returns:
            tuple: A tuple containing the following elements:
                - particle_features (torch.Tensor): The normalized particle features of the item.
                - subjets (torch.Tensor): The processed subjet data of the item.
                - indices (torch.Tensor): The particle indices for each subjet.
                - subjet_mask (torch.Tensor): The subjet mask data of the item.
                - particle_mask (torch.Tensor): The particle mask data of the item.
        """
        if self.debug:
            check_raw_data(self.subjets, jet_index=idx)  # -- Debug statment
            print(f"\nFetching item {idx} from dataset")

        particle_feature_names = ['part_deta', 'part_dphi', 'part_pt_log', 'part_e_log']
        particle_features = np.stack([self.particles[name][idx] for name in particle_feature_names])
        if self.debug:
            print("particle features shape", particle_features.shape)
        subjets = {name: self.subjets[name][idx] for name in self.subjets.keys()}
        if self.debug:
            print("subjets", subjets)

        if self.debug:
            inspect_indices(subjets) # -- Debug statment

        subjets, indices, subjet_mask, particle_mask = self.process_subjets(subjets)

        print("Normalizing features")
        particle_features = torch.from_numpy(particle_features)
        if self.config:
            particle_features = normalize_features(particle_features, particle_feature_names, self.config)
        else:
            print("No config provided for normalization")
        
        if self.transform:
            print("Applying transform to features")
            particle_features = self.transform(particle_features)
        
        # print(f"Returning data for item {idx}")
        # print(f"Features shape: {features.shape}")
        # print(f"Subjets shape: {subjets.shape}")
        # print(f"Subjet mask shape: {subjet_mask.shape}")
        # print(f"Particle mask shape: {particle_mask.shape}")
        # print(f"Subjets data: {subjets}")
        # print(f"Subjet mask data: {subjet_mask}")
        # print(f"Particle mask data: {particle_mask}")

        return particle_features, subjets, indices, subjet_mask, particle_mask

    def process_subjets(self, subjets):
        """
        Processes subjets to create tensor representations and masks.

        Parameters:
        - subjets (dictionary): a dictionary with keys ['subjet_pt', 'subjet_eta', 'subjet_phi', 'subjet_num_ptcls', 'particle_indices']. 
            each item is a numpy array of shape (N_subjets, ) or (N_subjets, N_particles) for the last key

        Returns:
        - tuple: (subjets, subjet_mask, particle_mask)
            where `subjets` is the tensor representation of subjets,
            `subjet_mask` is the mask for subjets,
            and `particle_mask` is the mask for particles.
        """
        if self.debug:
            print("Processing subjets")

        max_len = subjets['particle_indices'].shape[-1]
        if self.debug:
            print(f"Max length of indices in subjets: {max_len}")
        subjet_mask = torch.tensor(subjets['subjet_num_ptcls'] != 0, dtype=torch.float32)
        
        particle_mask = torch.tensor(subjets['particle_indices'] != -1, dtype=torch.float32)

        features = torch.stack([torch.from_numpy(subjets[name]) for name in ['subjet_pt', 'subjet_eta', 'subjet_phi', 'subjet_num_ptcls']], dim=0)
        features = features.t()
        
        indices = torch.from_numpy(subjets['particle_indices'])
        if self.debug:
            print(f"Subjet indices shape: {indices.shape}")  # (N_subjets, N_particles)
            
            print(f"Final processed subjets shape: {features.shape}") # (N_subjets, 4)
            print(f"Final subjet mask shape: {subjet_mask.shape}") # (N_subjets, ) same as 'subjet_num_ptcls'
            print(f"Final particle mask shape: {particle_mask.shape}") # (N_subjets, N_particles) same as 'particle_indices'
        
        return features, indices, subjet_mask, particle_mask


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        print("Initializing Attention module")
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        print(f"Attention forward pass with input shape: {x.shape}")
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        print(f"Attention output shape: {x.shape}")
        return x

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        print("Initializing MLP module")
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        print(f"MLP forward pass with input shape: {x.shape}")
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        print(f"MLP output shape: {x.shape}")
        return x

class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        print("Initializing Block module")
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = nn.Identity() if drop_path <= 0 else nn.Dropout(drop_path)
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        print(f"Block forward pass with input shape: {x.shape}")
        y = self.attn(self.norm1(x))
        x = x + self.drop_path(y)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        print(f"Block output shape: {x.shape}")
        return x

class JetsTransformer(nn.Module):
    def __init__(self,
        num_subjets=20, num_features=4, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True, qk_scale=None, drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, norm_layer=nn.LayerNorm, init_std=0.02,
        **kwargs
    ):
        super().__init__()
        self.num_features = num_features
        self.embed_dim = embed_dim 
        self.num_heads = num_heads

        self.subjet_embed = nn.Linear(num_features, embed_dim)
        self.num_subjets = num_subjets

        self.pos_embed = None  # We'll create this in the forward pass

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        self.init_std = init_std
        self.apply(self._init_weights)
        self.fix_init_weight()

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
    
    def fix_init_weight(self):
        def rescale(param, layer_id):
            param.div_(torch.sqrt(torch.tensor(2.0 *layer_id)))

        for layer_id, layer in enumerate(self.blocks):
            rescale(layer.attn.proj.weight.data, layer_id + 1)
            rescale(layer.mlp.fc2.weight.data, layer_id + 1)

    def forward(self, x, particle_indices, subjet_mask):
        print(f"JetsTransformer forward pass with input shape: {x.shape}")
        B, N, C = x.shape
        assert C == self.num_features, f"Input features {C} don't match expected features {self.num_features}"

        x = x.to(self.subjet_embed.weight.dtype)

        # Apply subjet embedding
        x = self.subjet_embed(x)
        

        if self.pos_embed is None or self.pos_embed.shape[1] != N:
            self.pos_embed = create_pos_emb(self.embed_dim, x).to(x.device)
        x = x + self.pos_embed

        # Apply subjet mask
        x = x * subjet_mask.unsqueeze(-1)
        

        # Apply transformer blocks
        for blk in self.blocks:
            x = blk(x)

        if self.norm is not None:
            x = self.norm(x)

        return x
            

class JetsTransformerPredictor(nn.Module):
    def __init__(self, num_features, embed_dim, predictor_embed_dim, depth, num_heads, mlp_ratio, qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm):
        super().__init__()
        print("Initializing JetsTransformerPredictor module")
        self.predictor_embed = nn.Linear(embed_dim, predictor_embed_dim, bias=True)
        self.predictor_blocks = nn.ModuleList([Block(dim=predictor_embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate, drop_path=drop_path_rate, norm_layer=norm_layer) for i in range(depth)])
        self.predictor_norm = norm_layer(predictor_embed_dim)
        self.predictor_proj = nn.Linear(predictor_embed_dim, 8 * 30, bias=True)  # Match target dimensions
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x, masks_x, masks):
        print(f"JetsTransformerPredictor forward pass with input shape: {x.shape}")
        x = self.predictor_embed(x)
        for blk in self.predictor_blocks:
            x = blk(x)
        x = self.predictor_proj(x)
        print(f"JetsTransformerPredictor output shape: {x.shape}")
        return x.view(x.size(0), x.size(1), 8, 30)  # Reshape to match target_repr shape

class JJEPA(nn.Module):
    def __init__(self, input_dim, embed_dim, depth, num_heads, mlp_ratio, dropout=0.1, use_predictor=True):
        super(JJEPA, self).__init__()
        print(f"Initializing JJEPA module with input_dim={input_dim}, embed_dim={embed_dim}")
        self.use_predictor = use_predictor
        self.context_transformer = JetsTransformer(num_features=input_dim, embed_dim=embed_dim, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=dropout)
        self.target_transformer = JetsTransformer(num_features=input_dim, embed_dim=embed_dim, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=dropout)
        if self.use_predictor:
            self.predictor_transformer = JetsTransformerPredictor(num_features=input_dim, embed_dim=embed_dim, predictor_embed_dim=embed_dim//2, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio, drop_rate=dropout)

        self.input_check = DimensionCheckLayer("Model Input", 3)
        self.context_check = DimensionCheckLayer("After Context Transformer", 3)
        self.predictor_check = DimensionCheckLayer("After Predictor", 3)

    def forward(self, context, target, particle_indices, subjet_mask):
        print(f"JJEPA forward pass with context shape: {context.shape} and target shape: {target.shape}")
        print(f"particle_indices shape: {particle_indices.shape}, subjet_mask shape: {subjet_mask.shape}")
        
        context = self.input_check(context)
        target = self.input_check(target)
        
        context_repr = self.context_transformer(context, particle_indices, subjet_mask)
        target_repr = self.target_transformer(target, particle_indices, subjet_mask)

        context_repr = self.context_check(context_repr)
        if self.use_predictor:
            pred_repr = self.predictor_transformer(context_repr, None, None)
            pred_repr = self.predictor_check(pred_repr)
            print(f"JJEPA output - pred_repr shape: {pred_repr.shape}, context_repr shape: {context_repr.shape}, target_repr shape: {target_repr.shape}")
            return pred_repr, context_repr, target_repr

        print(f"JJEPA output - context_repr shape: {context_repr.shape}, target_repr shape: {target_repr.shape}")
        return context_repr, target_repr

def train_step(model, particle_features, subjets, particle_indices, subjet_mask, particle_mask, optimizer, device, step):
    print(f"\nStarting training step {step}")

    batch_size, num_subjets = subjets.size()[:2]
    print(f"Input shapes - Particle features: {particle_features.shape}, Subjets: {subjets.shape}, Particle indices: {particle_indices.shape}")

    context_masks, target_masks = create_random_masks(batch_size, num_subjets, device=device)
    print(f"Context masks shape: {context_masks.shape}, Target masks shape: {target_masks.shape}")

    particle_features = particle_features.to(device)
    subjets = subjets.to(device)
    particle_indices = particle_indices.to(device)
    subjet_mask = subjet_mask.to(device)
    context_masks = context_masks.to(device)
    target_masks = target_masks.to(device)
    particle_mask = particle_mask.to(device)

    context_subjets = subjets * context_masks.unsqueeze(-1)
    target_subjets = subjets * target_masks.unsqueeze(-1)

    optimizer.zero_grad()

    print("Forwarding through model")
    pred_repr, context_repr, target_repr = model(context_subjets, target_subjets, particle_indices, subjet_mask)

    print(f"Predicted representation shape: {pred_repr.shape}")
    print(f"Target representation shape: {target_repr.shape}")

    combined_mask = target_masks.unsqueeze(-1).unsqueeze(-1) * subjet_mask.unsqueeze(-1).unsqueeze(-1)

    pred_repr = pred_repr.to(device)
    target_repr = target_repr.to(device)

    print("Calculating loss")
    loss = F.mse_loss(pred_repr * combined_mask, target_repr * combined_mask)
    print(f"Calculated loss: {loss.item()}")

    loss.backward()
    optimizer.step()

    if step % 500 == 0:
        print(f"Context representation shape: {context_repr.shape}")
        print(f"Target representation shape: {target_repr.shape}")

    return loss.item()


def get_1d_sincos_pos_emb(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega   # (D/2,)

    pos = pos.reshape(-1)   # (M,)
    out = np.einsum('m,d->md', pos, omega)   # (M, D/2), outer product

    emb_sin = np.sin(out)  # (M, D/2)
    emb_cos = np.cos(out)  # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


# position = torch.arange(max_len).unsqueeze(1)
# div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
# pe = torch.zeros(max_len, 1, d_model)
# pe[:, 0, 0::2] = torch.sin(position * div_term)
# pe[:, 0, 1::2] = torch.cos(position * div_term)
# self.register_buffer('pe', pe)

def create_pos_emb(emb_dim, ftrs):
    """
    Create positional embeddings for the subjets.
    
    Args:
    emb_dim (int): Embedding dimension
    ftrs (torch.Tensor): Tensor of shape [N_sj, N_ptcl_ftr, N_ptcl]
    
    Returns:
    torch.Tensor: Positional embeddings of shape [N_sj, emb_dim]
    """
    B, N_sj, _, _ = ftrs.size()
    
    ptcl_eta = ftrs[:, :, 1]
    ptcl_phi = ftrs[:, :, 2]

    sj_eta = torch.mean(ptcl_eta, axis=-1)
    sj_phi = torch.mean(ptcl_phi, axis=-1)

    # Process phi to impose physical distance
    sj_phi_star = torch.sin(sj_phi/2)
    
    # Shift eta to avoid negative positions
    sj_eta += 3
    
    # Calculate embedding
    emb_eta = get_1d_sincos_pos_emb(emb_dim//2, sj_eta.cpu().numpy())
    emb_phi_star = get_1d_sincos_pos_emb(emb_dim//2, sj_phi_star.cpu().numpy())
    
    emb = np.concatenate([emb_eta, emb_phi_star], axis=1)  # (H*W, D)
    return torch.from_numpy(emb).float()



def create_random_masks(batch_size, num_subjets, device, context_scale=0.7):
    """
    Create random masks for context and target subjets.
    
    Args:
        batch_size (int): Number of jets in the batch.
        num_subjets (int): Number of subjets in each jet.
        device (torch.device): The device to create the masks on.
        context_scale (float): Proportion of subjets to include in the context.
    
    Returns:
        tuple: (context_masks, target_masks), each of shape (batch_size, num_subjets)
    """
    print(f"Creating random masks with batch_size={batch_size}, num_subjets={num_subjets}")

    indices = torch.argsort(torch.rand(batch_size, num_subjets, device=device).argsort(dim=1))
    
    context_size = int(num_subjets * context_scale)

    context_mask = torch.zeros(batch_size, num_subjets, device=device)
    target_mask = torch.zeros(batch_size, num_subjets, device=device)

    context_mask.scatter_(1, indices[:, :context_size], 1)
    target_mask.scatter_(1, indices[:, context_size:], 1)

    return context_mask, target_mask

def visualize_training_loss(train_losses):
    print("Visualizing training loss")
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


if __name__ == "__main__":
    print("Starting main program")
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)

    try:
        print("Loading dataset")
        train_dataset = JetDataset("../data/val/val_20_30_new.h5", num_jets=1000, config=None)
    except Exception as e:
        print(f"Error loading dataset: {e}")

    print("Creating DataLoader")
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    print("Initializing model")
    model = JJEPA(input_dim=4, embed_dim=512, depth=12, num_heads=8, mlp_ratio=4.0, dropout=0.1).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.04)

    num_epochs = 10
    train_losses = []

    print(f"Starting training for {num_epochs} epochs")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", leave=True, position=0)

        for step, (particle_features, subjets, particle_indices, subjet_mask, particle_mask) in enumerate(train_loader):
            particle_features = particle_features.to(device)
            subjets = subjets.to(device)
            particle_indices = particle_indices.to(device)
            subjet_mask = subjet_mask.to(device)
            particle_mask = particle_mask.to(device)

            loss = train_step(model, particle_features, subjets, particle_indices, subjet_mask, particle_mask, optimizer, device, step)
            total_loss += loss

            progress_bar.set_postfix(loss=loss)
            progress_bar.update(1)

            if step % 100 == 0:
                print(f"\nEpoch {epoch+1}, Step {step}, Loss: {loss:.4f}")

        progress_bar.close()

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

    print("Training completed")
    print("Visualizing training loss")
    visualize_training_loss(train_losses)

    print("Saving model")
    torch.save(model.state_dict(), 'ijepa_model.pth')

    print("Model saved.")

Starting main program
Using device: cuda
Loading dataset
Initializing JetDataset with file: ../data/val/val_20_30.h5
Error loading dataset: [Errno 2] Unable to synchronously open file (unable to open file: name = '../data/val/val_20_30.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Creating DataLoader


NameError: name 'train_dataset' is not defined